In [6]:
import scanpy as sc
import scvelo as scv
import scarches as sca
import numpy as np
import pandas as pd
from velovi import preprocess_data
import scarches as sca

In [7]:
use_ontology=True
dataset="pancreas"

input_adata_file = "data/pancreas/full_processed_data.h5ad"

if use_ontology:
    output_adata_file = "data/pancreas/processed_ontology.h5ad"
else:
    output_adata_file = "pancreas_data_annotations.h5ad"

In [8]:
import scanpy as sc
adata=sc.read(input_adata_file)

In [9]:
if dataset=="pancreas":
    adata.obs["day"] = adata.obs.day.map(lambda day: day.replace(" (reference)", "").replace(" (query)", ""))
    adata=adata[adata.obs["day"]=="15.5"]

adata.layers["spliced"]=adata.layers["spliced"].todense()
adata.layers["unspliced"]=adata.layers["unspliced"].todense()

In [10]:
scv.pp.filter_and_normalize(adata, min_shared_counts=30, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

adata = preprocess_data(adata)

Filtered out 21583 genes that are detected 30 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:58) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:04) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
computing velocities
    finished (0:00:01) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [ ]:
if not use_ontology:

    data_path='data/annotations/'
    sca.utils.add_annotations(adata, [data_path+'PanglaoDB_markers_27_Mar_2020.gmt',data_path+'reactome.gmt'], min_genes=5, clean=True)
    #Remove all genes not present in annotations
    adata._inplace_subset_var(adata.varm['I'].sum(1)>0)

    #Filter out terms with less than 12 genes 
    select_terms = adata.varm['I'].sum(0)>12
    adata.uns['terms'] = np.array(adata.uns['terms'])[select_terms].tolist()
    adata.varm['I'] = adata.varm['I'][:, select_terms]

In [ ]:
adata.write_h5ad(output_adata_file)

In [ ]:
adata.n_obs

Latent Analysis